<a href="https://colab.research.google.com/github/cindyzhxng/2406-continuum-robot/blob/main/Motor%20Resolution%20and%20Torque%20Speccing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

NOTE: All units are metric.

# Motor Resolution

Standard stepper motors can increment the angle at [0.9 or 1.8 degrees](https://dovermotion.com/resources/motion-control-handbook/resolution/). The steppers from [our supplier of interest](https://www.omc-stepperonline.com/closed-loop-stepper-motor) increment at 1.8 degrees. This means a resolution of 360degrees/1.8degrees steps/revolution = **200 steps per revolution** Considering that our rotational maximum radius (i.e. robot length, 20cm) will be much larger than our pinion radius (45/2 cm), and assuming we use the same gear ratio like U of T for both our rotational and translational stages (before pinion), we can conclude that if rotation resolution is sufficient, translational resolution must be too.

Given that in the literature, the robot tip can travel with [accuracy of 1.5-3% of robot length](https://ieeexplore.ieee.org/document/9981719/references#references), we want to design the motors such that if the tubes were rigid, the smallest possible change in tangential distance travelled by the tip would ideally be at least ~0.2% of robot length, (2E-3)xL. Tangential resolution of the tip is (minimum achievable angular increment) x L. Thus, we want (minimum achievable angular increment)xL < 1E-3xL or simplified, minimum achievable angular increment < 2E-3 radians.


In [ ]:
required_resolution = 2*np.pi/2E-3
print("Achieving a resolution <.2% is equivalent to a resolution of", required_resolution, "steps per revolution.")
stepper_resolution = 360/1.8
print("Stepper motors that 1.8 degrees per full step have a resolution of", stepper_resolution, "steps per revolution.")
required_resolution_gear_factor = required_resolution/stepper_resolution
microsteps_per_step = 8
print("Assuming the stepper runs 8 microsteps per full step, achieving",
      required_resolution, "steps per revolution requires gear ratio of",
      required_resolution/(stepper_resolution*microsteps_per_step))

Achieving a resolution <.2% is equivalent to a resolution of 3141.592653589793 steps per revolution.
Stepper motors that 1.8 degrees per full step have a resolution of 200.0 steps per revolution.
Assuming the stepper runs 8 microsteps per full step, achieving 3141.592653589793 steps per revolution requires gear ratio of 1.9634954084936205


The microstepping by a factor of 8 was chosen taking into account resolution, torque, and accuracy needs.

Here we define a function that takes in gear ratio, and outputs the resolution (in percentage of minimum tip deviation compared to robot length - e.g. required is <.2%), assuming microstep of eight.

In [ ]:
def resolution_percent(gear_ratio):
  steps_per_revolution = stepper_resolution*microsteps_per_step*gear_ratio
  percent = 2*np.pi/steps_per_revolution * 100
  return percent

# Total Torque Equation:

TP = (TL+ TJ + TF) * 1.2

The torque due to inertia (TJ) is the torque required to accelerate the load from standstill or from a lower speed to a higher speed. This can be calculated by tak- ing the product of load inertia, including the rotor inertia and load acceleration.

Torque due to Inertia Equation:
TJ = (JL + M) * α

where:

JL+M is the sum of the load and rotor inertia and a is the required acceleration
The mechanical system coupled to the motor shaft determines the load torque and the frictional torque.

Total torque required include:


*   Load torque TL: the resistive torque presented by the external load itself, excluding inertia and friction. This torque arises from the mechanical resistance or requirements of the specific application.


>    For our case, the load torque is the torque required to push upon the ground and move the robot. Recall that the robot moves due to the reaction force from friction (e.g., the wheel pushes on the ground, and the friction force on the ground pushes the robot). This force is always present, in practice, to keep the robot moving at a constant speed since air friction and other dissipative forces will slow down the robot eventually.

*   Torque due to inertia TJ: torque required to accelerate the load from a lower speed to a higher speed. This depends on the inertia of the rotor and the wheel.
*   Torque to overcome friction TF: torque required to overcome dissipative forces at the motor-wheel system such as: air resistance, bearing friction, etc.

#Tube Rotation Motor - Required Torque

How much torque does our motor for rotating the tubes have to handle?

Considering turning up tube fully extended, turned 90 degrees from axis, in horizontal plane (maximal torque situation). Instead of finding the mass of three separate hollow tubes, we consider the outermost tube filled in with nitinol for a simplified overestimation of tube mass:

**Maximum Load Torque due to weight of tubes and camera:**

In [ ]:
outer_tube_diameter = 4.627E-3
tube_length = 20E-2
outer_tube_volume = np.pi*(outer_tube_diameter/2)**2 * tube_length
nitinol_density = 6450
filled_tube_mass = outer_tube_volume*nitinol_density
camera_mass = 10.2E-3
gravity = 9.81
load_torque = filled_tube_mass * gravity * tube_length/2 + camera_mass * gravity * tube_length
print("The maximum load torque due to camera and tube weight for tube rotation is", load_torque, "Nm")

The maximum load torque due to camera and tube weight for tube rotation is 0.041291222803014206 Nm


**Maximum Acceleration Torque**

In [ ]:
angular_rotation_speed = 2*np.pi
acceleration_time = 1.0/3
angular_acceleration = angular_rotation_speed/acceleration_time
tubes_moment_of_inertia = 1.0/3 * filled_tube_mass * tube_length**2
camera_moment_of_inertia = camera_mass*tube_length**2
acceleration_torque = (tubes_moment_of_inertia + camera_moment_of_inertia)*angular_acceleration
print("The acceleration torque is", acceleration_torque, "Nm")

The acceleration torque is 0.013142149368076575 Nm


Instead of calculating the frictional torque (between tubes) and the load torque due to inner tube torsion, we simplify our calculations using a high factor of safety on the load torque due to weight and acceleration torque:

In [ ]:
safety_factor = 5
tube_rotation_torque = 5 * (load_torque+acceleration_torque)
print("Therefore, we approximate that the torque our motors for rotating the tubes should be able to deliver after gear ratio is", tube_rotation_torque, "Nm")

Therefore, we approximate that the torque our motors for rotating the tubes should be able to deliver after gear ratio is 0.2721668608554539 Nm


## Tube Translation Motor - Required v.s. Delivered Force at Rack and Pinion Interface


This one is a bit more complicated than tube rotation, because force required varies depending on the weight of the motors.

First, we make a function for required force that takes as parameter the motor mass:

In [ ]:
def required_force_at_rack_pinion(actuation_unit_mass):
    weight = actuation_unit_mass * gravity

    # Interface is pillow blocks and rails. pillow blocks surface made of iglide® J200
    # which has a friction coefficient <0.2 with Hard adonized aluminum (? "best" partner material according to same site, below...)
    # https://www.igus.eu/info/plain-bearings-j200-material-data#:~:text=Among%20all%20the%20iglidur%C2%AE,anodised%20aluminium%20assumes%20special%20importance.
    coefficient_of_friction = .2

    friction_force = coefficient_of_friction * weight
    return friction_force

def actuation_unit_mass(motor_mass):
    actuation_frame_volume = 0.00014
    plastic_parts_total_volume = actuation_frame_volume * 2 #an approximation
    resin_density = 1110
    plastic_parts_mass = plastic_parts_total_volume * resin_density
    pillow_block_mass = .02 # https://www.igus.ca/ContentData/Products/Downloads/drylin-w-pillow-block-wj200um-01-al.pdf
    total_mass = 2 * motor_mass + plastic_parts_mass + filled_tube_mass + 4*pillow_block_mass
    return total_mass

Because we are microstepping the motors, our holding torque is reduced using the formula, *incremental holding torque = (full stepping torque) x sin (90 degrees/ number of microsteps) *

In [ ]:
def incremental_holding_torque(full_stepping_torque,microsteps_per_step):
  return full_stepping_torque*np.sin((np.pi/2)/microsteps_per_step)

Here is a function that converts the motor torque to force at the pinion-gear interface:

In [ ]:
def force_after_gear_ratio(motor_torque, input_gear_diameter, output_gear_diameter, pinion_gear_diameter):
  output_torque = torque_after_gear_ratio(motor_torque, input_gear_diameter, output_gear_diameter)
  if pinion_gear_diameter == None:
    pinion_gear_diameter = output_gear_diameter
  return output_torque/(pinion_gear_diameter/2) # = (motor_torque*gear_ratio)/pinion_gear_radius

def torque_after_gear_ratio(motor_torque, input_gear_diameter, output_gear_diameter):
  return motor_torque*output_gear_diameter/input_gear_diameter # = motor_torque*gear_ratio

We now list the translational forces required and the forces provided for each motor of interest.

In [ ]:
def motor_speccing(motor_mass, motor_torque, input_gear_diameter, output_gear_diameter, motor_link, pinion_gear_diameter=None):
  print (motor_link)
  print("required resolution (minimum tip deviation to robot length): .2%")
  print("resolution after 8x microstep, gear ratio (minimum tip deviation to robot length):", resolution_percent(output_gear_diameter/input_gear_diameter),"%")
  one_axis_unit_mass = actuation_unit_mass(motor_mass)
  print ("required force at rack and pinion:", required_force_at_rack_pinion(one_axis_unit_mass))
  max_applicable_force = force_after_gear_ratio(motor_torque, input_gear_diameter, output_gear_diameter, pinion_gear_diameter)
  print ("maximum force applicable at rack and pinion:", max_applicable_force)
  print ("actual max acceleration:", max_applicable_force/one_axis_unit_mass, "m/s^2")
  print("max output torque:", torque_after_gear_ratio(motor_torque, input_gear_diameter, output_gear_diameter))
  print("required output torque for tube rotation (after SF=5):", tube_rotation_torque)

In [ ]:
motor_speccing(.123, incremental_holding_torque(0.054, microsteps_per_step), .01, .04, "https://www.omc-stepperonline.com/nema-8-closed-loop-stepper-motor-0-054-nm-7-65oz-in-encoder-1000ppr-4000cpr-8hs22-0804d-e1000")

https://www.omc-stepperonline.com/nema-8-closed-loop-stepper-motor-0-054-nm-7-65oz-in-encoder-1000ppr-4000cpr-8hs22-0804d-e1000
required resolution (minimum tip deviation to robot length): .2%
resolution after 8x microstep, gear ratio (minimum tip deviation to robot length): 0.09817477042468105 %
required force at rack and pinion: 1.2919592456060283
maximum force applicable at rack and pinion: 2.106975477774185
actual max acceleration: 3.199703010332838 m/s^2
max output torque: 0.0421395095554837
required output torque for tube rotation (after SF=5): 0.2721668608554539


In [ ]:
motor_speccing(.25, incremental_holding_torque(0.16, microsteps_per_step), .012, .04, "https://www.omc-stepperonline.com/nema-17-closed-loop-stepper-motor-16ncm-22-7oz-in-with-magnetic-encoder-1000ppr-4000cpr-17hs08-1004-me1k")

https://www.omc-stepperonline.com/nema-17-closed-loop-stepper-motor-16ncm-22-7oz-in-with-magnetic-encoder-1000ppr-4000cpr-17hs08-1004-me1k
required resolution (minimum tip deviation to robot length): .2%
resolution after 8x microstep, gear ratio (minimum tip deviation to robot length): 0.11780972450961723 %
required force at rack and pinion: 1.7903072456060283
maximum force applicable at rack and pinion: 5.202408587096753
actual max acceleration: 5.7013262237167925 m/s^2
max output torque: 0.10404817174193506
required output torque for tube rotation (after SF=5): 0.2721668608554539


**This one is small with a high safety factor (~7). We want a high safety factor because our calculations do not take into account resistance due to pre-bent concentric tubes, and we can't waste time with the wrong order:**

In [ ]:
motor_speccing(.3, incremental_holding_torque(.45,microsteps_per_step),0.012, 0.04, "https://www.omc-stepperonline.com/nema-17-closed-loop-stepper-motor-45ncm-64oz-in-with-magnetic-encoder-1000ppr-4000cpr-17hs15-1504-me1k")

https://www.omc-stepperonline.com/nema-17-closed-loop-stepper-motor-45ncm-64oz-in-with-magnetic-encoder-1000ppr-4000cpr-17hs15-1504-me1k
required resolution (minimum tip deviation to robot length): .2%
resolution after 8x microstep, gear ratio (minimum tip deviation to robot length): 0.11780972450961723 %
required force at rack and pinion: 1.9865072456060286
maximum force applicable at rack and pinion: 14.631774151209619
actual max acceleration: 14.451264121070645 m/s^2
max output torque: 0.2926354830241924
required output torque for tube rotation (after SF=5): 0.2721668608554539


In [ ]:
motor_speccing(.3, incremental_holding_torque(.45,microsteps_per_step),0.012, 0.04, "https://www.omc-stepperonline.com/nema-17-closed-loop-stepper-motor-45ncm-64oz-in-with-magnetic-encoder-1000ppr-4000cpr-17hs15-1504-me1k", .04501)

https://www.omc-stepperonline.com/nema-17-closed-loop-stepper-motor-45ncm-64oz-in-with-magnetic-encoder-1000ppr-4000cpr-17hs15-1504-me1k
required resolution (minimum tip deviation to robot length): .2%
resolution after 8x microstep, gear ratio (minimum tip deviation to robot length): 0.11780972450961723 %
required force at rack and pinion: 1.9865072456060286
maximum force applicable at rack and pinion: 13.003131882879021
actual max acceleration: 12.842714171135876 m/s^2
max output torque: 0.2926354830241924
required output torque for tube rotation (after SF=5): 0.2721668608554539


In [ ]:
motor_speccing(.5, incremental_holding_torque(.65,8), 0.012, 0.04, "https://www.omc-stepperonline.com/nema-17-closed-loop-stepper-motor-65ncm-92oz-in-with-magnetic-encoder-1000ppr-4000cpr-17hs24-2104-me1k")

https://www.omc-stepperonline.com/nema-17-closed-loop-stepper-motor-65ncm-92oz-in-with-magnetic-encoder-1000ppr-4000cpr-17hs24-2104-me1k
required resolution (minimum tip deviation to robot length): .2%
resolution after 8x microstep, gear ratio (minimum tip deviation to robot length): 0.11780972450961723 %
required force at rack and pinion: 2.7713072456060286
maximum force applicable at rack and pinion: 21.134784885080556
actual max acceleration: 14.962775423141574 m/s^2
max output torque: 0.42269569770161114
required output torque for tube rotation (after SF=5): 0.2721668608554539


For comparison, here is U of T's specs:

In [ ]:
motor_speccing(.053, .26, 0.01, 0.04, "https://store.tmotor.com/goods-438-Antigravity+MN4004+KV300+-+2PCSSET.html", 0.045)

https://store.tmotor.com/goods-438-Antigravity+MN4004+KV300+-+2PCSSET.html
required resolution (minimum tip deviation to robot length): .2%
resolution after 8x microstep, gear ratio (minimum tip deviation to robot length): 0.09817477042468105 %
required force at rack and pinion: 1.0172792456060284
maximum force applicable at rack and pinion: 46.22222222222223
actual max acceleration: 89.14759678005034 m/s^2
max output torque: 1.04
required output torque for tube rotation (after SF=5): 0.2721668608554539
